In [ ]:
# mount my Google Drive to save the notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install libraries
!pip install -q diffusers transformers accelerate peft bitsandbytes kaggle

In [ ]:
# Kaggle API
from google.colab import files
files.upload() # upload kaggle.json API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download Tom and Jerry dataset
!kaggle datasets download -d balabaskar/tom-and-jerry-image-classification -p /content/tom_and_jerry

In [ ]:
# unzip the dataset
!unzip -q /content/tom_and_jerry/tom-and-jerry-image-classification.zip -d /content/tom_and_jerry_dataset

In [ ]:
# inspect dataset to verify and start data preparation
import os
dataset_path = '/content/tom_and_jerry_dataset/tom_and_jerry/tom_and_jerry'
print(os.listdir(dataset_path))

In [ ]:
# get all images from the dataset
import glob
from PIL import Image

image_paths = []
style_data_root = '/content/tom_and_jerry_dataset/tom_and_jerry/tom_and_jerry/'

# collect images from all subfolders -> goes through all(files and folders) in the style_data_root directory
for folder_name in os.listdir(style_data_root):
  folder_path = os.path.join(style_data_root, folder_name)
  if os.path.isdir(folder_path):
    # add all files to image_paths list
    image_paths.extend(glob.glob(os.path.join(folder_path, '*.jpg')))
    image_paths.extend(glob.glob(os.path.join(folder_path, '*.png'))) # just in case but I think its only .jpg files
print(f"Found  {len(image_paths)} images to use for training.")

In [ ]:
# load the base model = Stable Diffusion v1.5
import torch
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model, PeftModel

model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda"

# load pipeline with 4-bit quantization
pipeline = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16, # try full 32bits precision later -> or rent a better GPU
    load_in_4bit=True, # model's weights and biases are loaded and stored using only 4 bits per value
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
pipeline.to(device)

pipeline.text_encoder.to(dtype=torch.float16, device=device)
pipeline.vae.to(dtype=torch.float16, device=device)

print("Base model loaded.")

In [ ]:
# configure LoRA for UNet
from peft import LoraConfig, get_peft_model

# configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32, # often 2*r
    target_modules=["to_q", "to_k", "to_v", "to_out.0", "proj_in", "proj_out"],
    lora_dropout=0.05,
    bias="none",
)

# add LoRA adapters to the UNet -> making the UNet trainable, not the whole pipeline
unet = pipeline.unet
unet_lora = get_peft_model(unet, lora_config)
# verify LoRA application
unet_lora.print_trainable_parameters()